# 🏥 Heart Disease Prediction - V16 Ensemble

## **6-Model Balanced Ensemble with Full Domain Feature Engineering**

### Key Features:
- **3 algorithms:** LightGBM + CatBoost + XGBoost
- **2 seeds each:** [42, 123] for diversity
- **13 domain-specific features** from medical knowledge
- **Target encoding:** 7 categorical features with smoothing=30
- **Meta-learner:** Logistic Regression on OOF predictions

### Results:
- **Kaggle AUC:** 0.95359
- **Rank:** 1038/3981 (26.1%) 🎯
- **Status:** ⭐ LATEST - Best Algorithm Diversity!

## Step 1: Import Libraries & Load Data

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# Load datasets (local paths for testing)
train_df = pd.read_csv('train.csv')
test_df  = pd.read_csv('test.csv')

target_column = 'Heart Disease'
X_train = train_df.drop([target_column, 'id'], axis=1)
y_train = train_df[target_column].map({'Absence': 0, 'Presence': 1})
X_test  = test_df.drop('id', axis=1)
test_ids = test_df['id'].values

# Fill missing values
X_train = X_train.fillna(X_train.median()).reset_index(drop=True)
X_test  = X_test.fillna(X_test.median()).reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

print(f"Train: {X_train.shape} | Test: {X_test.shape}")

## Step 2: Domain Feature Engineering

Create 13 medical domain-specific features capturing cardiac patterns.

In [ ]:
def engineer_features(df):
    """Create domain-specific medical features"""
    df = df.copy()
    
    # Chest pain indicators
    df['cp_asymptomatic']  = (df['Chest pain type'] == 4).astype(int)
    df['cp_angina']        = df['Chest pain type'] * df['Exercise angina']
    
    # ST segment features (cardiac risk)
    df['st_significant']   = (df['ST depression'] > 2.0).astype(int)
    df['st_slope_combo']   = df['ST depression'] * df['Slope of ST']
    df['angina_st']        = df['Exercise angina'] * df['ST depression']
    
    # Heart rate analysis
    df['max_hr_ratio']     = df['Max HR'] / (220 - df['Age'])  # Estimated max HR ratio
    df['hr_reserve']       = (220 - df['Age']) - df['Max HR']  # Heart rate reserve
    
    # Vascular and blood indicators
    df['has_vessels']      = (df['Number of vessels fluro'] > 0).astype(int)
    df['vessels_thal']     = df['Number of vessels fluro'] * df['Thallium']
    df['high_bp']          = (df['BP'] > 140).astype(int)
    df['high_chol']        = (df['Cholesterol'] > 240).astype(int)
    
    # Thalassemia patterns
    df['thal_reversible']  = (df['Thallium'] == 7).astype(int)
    df['thal_fixed']       = (df['Thallium'] == 6).astype(int)
    
    # Age-based risk
    df['age_male']         = df['Age'] * df['Sex']
    df['fbs_age']          = df['FBS over 120'] * df['Age']
    
    # Composite risk score
    df['risk_score'] = (
        df['cp_asymptomatic'] +
        df['st_significant'] +
        df['has_vessels'] +
        df['thal_reversible'] +
        df['Exercise angina'] +
        (df['ST depression'] > 1.0).astype(int) +
        (df['max_hr_ratio'] < 0.7).astype(int)
    )
    
    # Age grouping for categorical patterns
    df['age_group'] = pd.cut(
        df['Age'], bins=[0,45,55,65,100], labels=[0,1,2,3]
    ).astype(int)
    
    return df

X_train = engineer_features(X_train)
X_test  = engineer_features(X_test)

print(f"Features after engineering: {X_train.shape[1]}")

## Step 3: Target Encoding with Smoothing

Apply target encoding inside each CV fold to prevent data leakage.

In [ ]:
TE_COLS = ['Chest pain type', 'Thallium', 'Slope of ST',
           'Number of vessels fluro', 'EKG results',
           'risk_score', 'age_group']

def target_encode(X_tr, y_tr, X_val, X_te, cols, smoothing=30):
    """Target encode features within fold to prevent leakage"""
    gm = y_tr.mean()  # Global mean
    X_tr_e, X_val_e, X_te_e = X_tr.copy(), X_val.copy(), X_te.copy()
    
    for col in cols:
        # Smoothed target encoding
        stats = y_tr.groupby(X_tr[col]).agg(['sum','count'])
        sm = (stats['sum'] + gm * smoothing) / (stats['count'] + smoothing)
        
        # Map and fill missing values
        X_tr_e[f'{col}_te']  = X_tr[col].map(sm).fillna(gm)
        X_val_e[f'{col}_te'] = X_val[col].map(sm).fillna(gm)
        X_te_e[f'{col}_te']  = X_te[col].map(sm).fillna(gm)
    
    return X_tr_e, X_val_e, X_te_e

print(f"Will target encode {len(TE_COLS)} features")

## Step 4: 5-Fold CV with 6 Models (3 Algorithms × 2 Seeds)

Train balanced ensemble with best algorithm diversity:
- **1 CV seed:** 42 (fixed)
- **2 model seeds:** 42, 123
- **3 algorithms:** LightGBM + CatBoost + XGBoost

In [ ]:
n_folds = 5
SEEDS   = [42, 123]
skf     = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

all_oof   = []
all_test  = []
all_scores = []

print(f"Training 6 models × {n_folds} folds")
print(f"Est. runtime: ~5.5 hours\n")

for seed in SEEDS:
    print(f"\n{'='*50}")
    print(f"Seed: {seed}")
    print(f"{'='*50}")

    # ======================== LightGBM ========================
    oof_lgb  = np.zeros(len(X_train))
    test_lgb = np.zeros(len(X_test))
    sc_lgb   = []

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        print(f"  LGB fold {fold+1}/5", end="")

        # Target encode within fold
        X_tr_e, X_val_e, X_te_e = target_encode(
            X_train.iloc[tr_idx], y_train.iloc[tr_idx],
            X_train.iloc[val_idx], X_test, TE_COLS)

        # LightGBM with optimized params
        m = lgb.LGBMClassifier(
            n_estimators=10000,
            learning_rate=0.01,
            subsample=0.7,
            subsample_freq=1,
            colsample_bytree=0.7,
            min_child_samples=50,
            reg_alpha=0.1,
            reg_lambda=5.0,
            num_leaves=31,
            min_split_gain=0.01,
            random_state=seed,
            n_jobs=-1,
            verbose=-1
        )
        m.fit(X_tr_e, y_train.iloc[tr_idx],
              eval_set=[(X_val_e, y_train.iloc[val_idx])],
              callbacks=[lgb.early_stopping(200, verbose=False),
                         lgb.log_evaluation(-1)])

        # Generate predictions
        oof_lgb[val_idx] = m.predict_proba(X_val_e)[:, 1]
        test_lgb += m.predict_proba(X_te_e)[:, 1] / n_folds
        
        fold_auc = roc_auc_score(y_train.iloc[val_idx], oof_lgb[val_idx])
        sc_lgb.append(fold_auc)
        print(f" → AUC: {fold_auc:.6f}")

    all_oof.append(oof_lgb)
    all_test.append(test_lgb)
    all_scores.append(sc_lgb)
    mean_auc = np.mean(sc_lgb)
    std_auc = np.std(sc_lgb)
    print(f"   LGB-s{seed}: {mean_auc:.6f} ± {std_auc:.6f}")

    # ======================== CatBoost ========================
    oof_cat  = np.zeros(len(X_train))
    test_cat = np.zeros(len(X_test))
    sc_cat   = []

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        print(f"  CAT fold {fold+1}/5", end="")

        # Target encode within fold
        X_tr_e, X_val_e, X_te_e = target_encode(
            X_train.iloc[tr_idx], y_train.iloc[tr_idx],
            X_train.iloc[val_idx], X_test, TE_COLS)

        # CatBoost with optimized params
        m = CatBoostClassifier(
            iterations=10000,
            depth=5,
            learning_rate=0.01,
            l2_leaf_reg=10,
            min_data_in_leaf=50,
            random_strength=1,
            bagging_temperature=0.5,
            random_state=seed,
            verbose=False,
            thread_count=-1,
            early_stopping_rounds=200,
            eval_metric='AUC'
        )
        m.fit(X_tr_e, y_train.iloc[tr_idx],
              eval_set=(X_val_e, y_train.iloc[val_idx]),
              verbose=False)

        # Generate predictions
        oof_cat[val_idx] = m.predict_proba(X_val_e)[:, 1]
        test_cat += m.predict_proba(X_te_e)[:, 1] / n_folds
        
        fold_auc = roc_auc_score(y_train.iloc[val_idx], oof_cat[val_idx])
        sc_cat.append(fold_auc)
        print(f" → AUC: {fold_auc:.6f}")

    all_oof.append(oof_cat)
    all_test.append(test_cat)
    all_scores.append(sc_cat)
    mean_auc = np.mean(sc_cat)
    std_auc = np.std(sc_cat)
    print(f"   CAT-s{seed}: {mean_auc:.6f} ± {std_auc:.6f}")

    # ======================== XGBoost ========================
    oof_xgb  = np.zeros(len(X_train))
    test_xgb = np.zeros(len(X_test))
    sc_xgb   = []

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        print(f"  XGB fold {fold+1}/5", end="")

        # Target encode within fold
        X_tr_e, X_val_e, X_te_e = target_encode(
            X_train.iloc[tr_idx], y_train.iloc[tr_idx],
            X_train.iloc[val_idx], X_test, TE_COLS)

        # XGBoost with optimized params
        m = xgb.XGBClassifier(
            n_estimators=10000,
            max_depth=4,
            learning_rate=0.01,
            subsample=0.7,
            colsample_bytree=0.7,
            min_child_weight=10,
            reg_alpha=0.1,
            reg_lambda=5.0,
            gamma=0.2,
            random_state=seed,
            n_jobs=-1,
            eval_metric='auc',
            early_stopping_rounds=200
        )
        m.fit(X_tr_e, y_train.iloc[tr_idx],
              eval_set=[(X_val_e, y_train.iloc[val_idx])],
              verbose=False)

        # Generate predictions
        oof_xgb[val_idx] = m.predict_proba(X_val_e)[:, 1]
        test_xgb += m.predict_proba(X_te_e)[:, 1] / n_folds
        
        fold_auc = roc_auc_score(y_train.iloc[val_idx], oof_xgb[val_idx])
        sc_xgb.append(fold_auc)
        print(f" → AUC: {fold_auc:.6f}")

    all_oof.append(oof_xgb)
    all_test.append(test_xgb)
    all_scores.append(sc_xgb)
    mean_auc = np.mean(sc_xgb)
    std_auc = np.std(sc_xgb)
    print(f"   XGB-s{seed}: {mean_auc:.6f} ± {std_auc:.6f}")

print("\n" + "="*50)
print("TRAINING COMPLETE")
print("="*50)

## Step 5: Ensemble Comparison & Selection

Compare 3 ensemble methods and select the best:

In [ ]:
print("\n" + "="*50)
print("INDIVIDUAL MODEL PERFORMANCE")
print("="*50)

labels = [f'{m}-s{s}' for s in SEEDS for m in ['LGB','CAT','XGB']]
for i, label in enumerate(labels):
    mean_auc = np.mean(all_scores[i])
    std_auc = np.std(all_scores[i])
    print(f"{label:12s}: {mean_auc:.6f} ± {std_auc:.6f}")

print("\n" + "="*50)
print("ENSEMBLE METHODS")
print("="*50)

# ======================== Simple Average ========================
simple_oof  = np.mean(all_oof,  axis=0)
simple_test = np.mean(all_test, axis=0)
simple_auc  = roc_auc_score(y_train, simple_oof)
print(f"Simple Average OOF AUC: {simple_auc:.6f}")

# ======================== Rank Average ========================
def rank_avg(preds_list):
    """Percentile-based rank averaging"""
    r = np.zeros(len(preds_list[0]))
    for p in preds_list:
        r += pd.Series(p).rank(pct=True).values
    return r / len(preds_list)

rank_oof  = rank_avg(all_oof)
rank_test = rank_avg(all_test)
rank_auc  = roc_auc_score(y_train, rank_oof)
print(f"Rank Average OOF AUC:   {rank_auc:.6f}")

# ======================== Meta-model (Logistic Regression) ========================
meta = LogisticRegression(C=0.01, max_iter=1000, random_state=42)
mtr  = pd.DataFrame({f'm{i}': all_oof[i]  for i in range(len(all_oof))})
mte  = pd.DataFrame({f'm{i}': all_test[i] for i in range(len(all_test))})

meta.fit(mtr, y_train)
meta_oof  = meta.predict_proba(mtr)[:, 1]
meta_test = meta.predict_proba(mte)[:, 1]
meta_auc  = roc_auc_score(y_train, meta_oof)
print(f"Meta-model OOF AUC:     {meta_auc:.6f}")

print("\n" + "="*50)
print("BEST METHOD SELECTED")
print("="*50)

# ======================== Pick Best ========================
scores_dict = {
    'simple': (simple_auc, simple_test),
    'rank':   (rank_auc,   rank_test),
    'meta':   (meta_auc,   meta_test),
}
best_name = max(scores_dict, key=lambda k: scores_dict[k][0])
best_auc, best_test = scores_dict[best_name]
print(f"Best Method: {best_name.upper()}")
print(f"OOF AUC: {best_auc:.6f}")

## Step 6: Generate and Save Submission

In [ ]:
# Create submission
submission = pd.DataFrame({
    'id': test_ids,
    'Heart Disease': np.clip(best_test, 0, 1)
})

# Save to local path
submission.to_csv('submission.csv', index=False)

print("\n" + "="*50)
print("SUBMISSION SAVED")
print("="*50)
print(f"\nFile: submission.csv")
print(f"Shape: {submission.shape}")
print(f"\nFirst 10 rows:")
print(submission.head(10))
print(f"\nPrediction range: [{submission['Heart Disease'].min():.6f}, {submission['Heart Disease'].max():.6f}]")
print(f"\n✅ Ready for submission!")